In [1]:

import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

q = '''SELECT * FROM ebmdatalab.outlier_detection.chem_by_subpara_by_ccg_juntoaug17
-- exclude non-standard CCG codes:
WHERE SUBSTR(pct,1,1) NOT BETWEEN 'A' AND 'Z' -- first character numerical
AND SUBSTR(pct,3,1) BETWEEN 'A' AND 'Z'  -- last character alphabetical
'''

df1 = pd.io.gbq.read_gbq(q, GBQ_PROJECT_ID, dialect='standard',verbose=False)
# rows: pct, chemical, subpara, num, denom, ratio (num and denom are items not quantity)

q2 = '''SELECT DISTINCT chemical, chemical_code from ebmdatalab.hscic.bnf'''
chem = pd.io.gbq.read_gbq(q2, GBQ_PROJECT_ID, dialect='standard',verbose=False)

q3 = '''SELECT DISTINCT subpara, subpara_code from ebmdatalab.hscic.bnf'''
subp = pd.io.gbq.read_gbq(q3, GBQ_PROJECT_ID, dialect='standard',verbose=False)

In [104]:
# need to flag where ccgs have not prescribed any items of the denominator in order to clean the data. 

# Step 1: amend the datafrome to include a line for every CCG and every chemical and subparagraph.

# list all subpara-chemical combinations 
a = df1[["subpara", "chemical"]].drop_duplicates()

#list all ccgs
b = df1[["pct"]].drop_duplicates()

# cross join to make table of all CCGs and all subpara combinations 
a['tmp'] = 1
b['tmp'] = 1
c = b.merge(a, on="tmp").drop('tmp', axis=1) # 237,636 rows

# join to data - need to list every possible chemical against every CCG
data = c.merge(df1, how="left", on=["pct","subpara","chemical"])  # 237,636 rows
data


# Step 2: identify those with zero subparas
# subpara totals by ccg
subpara = df1[["pct","subpara","denom"]].groupby(["subpara","pct"]).max().reset_index() # 42,917 rows

#list all possible subparagraphs and all ccgs
a2 = df1[["subpara"]].drop_duplicates()
a2['tmp'] = 1

# cross join to CCGs to make table of all CCGs and all subpara combinations 
c2 = b.merge(a2, on="tmp").drop('tmp', axis=1) # 56,097 rows

# join to subpara data by ccg to identify subparas prescribed by each ccg.  
d = c2.merge(subpara,how="left", on=["subpara","pct"])

# for subparas never prescribed, replace NAs with zeros so that there is data present to indicate this
d = d.fillna(0)

# join back to original dataset
d2 = d.merge(data, how="left", on=["subpara","pct"], suffixes=("_subpara",""))
# check how many have zero denominators:
# data.loc[(data["denom_subpara"]==0)] # 19,665 rows 

# exclude combinations where denominators are zero THEN replace NAs with 0:
data2 = d2.loc[(d2["denom_subpara"]!=0)]
data2 = data2.fillna(0)
data2

,pct,subpara,denom_subpara,chemical,num,denom,ratio
0,05X,1104020,1305.0,1104020M0,30.0,1305.0,0.022989
1,05X,1104020,1305.0,1104020N0,126.0,1305.0,0.096552
2,05X,1104020,1305.0,1104020T0,1006.0,1305.0,0.770881
3,05X,1104020,1305.0,1104020W0,32.0,1305.0,0.024521
4,05X,1104020,1305.0,1104020Z0,77.0,1305.0,0.059004
5,05X,1104020,1305.0,1104020AC,0.0,0.0,0.000000
6,05X,1104020,1305.0,1104020AE,0.0,0.0,0.000000
7,05X,1104020,1305.0,1104020B0,0.0,0.0,0.000000
8,05X,1104020,1305.0,1104020Y0,0.0,0.0,0.000000
9,05X,1104020,1305.0,110402000,0.0,0.0,0.000000


In [105]:
#select columns of interest and get key stats
df2 = data2[["chemical","subpara", "ratio"]].groupby(["chemical","subpara"]).describe()
df2 = df2.unstack()
df2.columns = df2.columns.droplevel()
df2 = df2.reset_index()

#limit to chemicals prescribed by at least 10 CCGs??
df3 = df2#.loc[df2["count"]>9].reset_index()
df3["range"] = df3["max"] - df3["min"]
df3 = df3[["chemical","subpara","count","50%","min","max","range","std"]].rename(columns={"50%":"median"})

df3

,chemical,subpara,count,median,min,max,range,std
0,0101010C0,0101010,192.0,0.000000,0.000000,0.340426,0.340426,0.057828
1,0101010F0,0101010,192.0,0.000000,0.000000,0.185629,0.185629,0.013397
2,0101010G0,0101010,192.0,0.441441,0.000000,0.912442,0.912442,0.212878
3,0101010I0,0101010,192.0,0.000000,0.000000,0.294872,0.294872,0.044221
4,0101010J0,0101010,192.0,0.000000,0.000000,0.129032,0.129032,0.015764
5,0101010L0,0101010,192.0,0.000000,0.000000,0.411765,0.411765,0.055342
6,0101010N0,0101010,192.0,0.000000,0.000000,0.118644,0.118644,0.012905
7,0101010Q0,0101010,192.0,0.000000,0.000000,0.773087,0.773087,0.147875
8,0101010R0,0101010,192.0,0.240833,0.000000,0.864865,0.864865,0.172471
9,0101012B0,0101012,28.0,1.000000,0.956522,1.000000,0.043478,0.008217


In [18]:
# reshape data to put CCGs in columns
df5 = data2.pivot(index="chemical",columns='pct', values='ratio')

#sum numerators to find total volume for each chemical
num = pd.DataFrame(df1["num"].groupby(df1["chemical"]).sum()).reset_index()

#calculate kurtosis and skew for each chemical
import scipy.stats as stats
k = pd.Series(stats.kurtosis(df5, axis=1,nan_policy="omit"),name="kurtosis")
sk =  pd.Series(stats.skew(df5, axis=1,nan_policy="omit"),name="skew")

num["num centile"] = pd.qcut(num["num"], 10, labels=np.arange(1,11,1))
num

,chemical,num,num centile
0,0101010C0,724,3
1,0101010F0,31,1
2,0101010G0,19555,6
3,0101010I0,416,2
4,0101010J0,228,2
5,0101010L0,939,3
6,0101010N0,164,2
7,0101010Q0,2410,3
8,0101010R0,9243,5
9,0101012B0,996,3


In [19]:

#count non-zero values to indicate how many CCGs have prescribed each chemical. 
#count = pd.Series(df5.count(axis=1),name="CCG count")

# replace nulls with zeros to take into account CCGs prescribing none in the summary stats
'''df6 = df5#.fillna(0)
df6 = pd.DataFrame(df6.stack()).reset_index().rename(columns={0:"ratio"})
df6 = df6.groupby("chemical").describe().unstack()
df6 = df6.reset_index(col_level=1)
df6.columns = df6.columns.droplevel()
smry = df6[["50%", "min","max","std"]].rename(columns={"50%":"median","min":"abs_min","std":"std_inc_zeros","max":"max2"})
smry["abs_range"] = smry["max2"]- smry["abs_min"]'''


#compile all results together
result = pd.concat([df3, k, sk], axis=1).sort_values(by="kurtosis",ascending=False)
result = result.merge(num, on="chemical")
#result[["chemical","subpara","num","count","median","abs_min","min","max","range","abs_range", "std","std_inc_zeros","kurtosis","skew"]].round(2)
result = result[["chemical","subpara","num","num centile", "count","median","min","max","range","std","kurtosis","skew"]].round(2)

In [20]:
# Lookup chemical and subparagraph names
df4 = result.merge(chem, how="left", left_on = "chemical",right_on="chemical_code",suffixes=(""," name"))
df4 = df4.merge(subp, how="left", left_on = "subpara",right_on="subpara_code",suffixes=(""," name"))
#df3 = df3[["chemical","chemical name","subpara","subpara name","min","max","range","std"]]
df4 = df4[["chemical","chemical name","subpara","subpara name","num","num centile", "count","median","min","max","range", "std","kurtosis","skew"]].round(2)

In [38]:
# sort by range first
r1 = df4.loc[(df4["num centile"]>2)].sort_values(by=["range","kurtosis"],ascending=False).head(20)
# create a flag
r1["R"] = 1

# sort by kurtosis and limit to items with at least 1% range 
r2 = df4.loc[(df4["range"] >0.1) & (df4["num centile"]>2)].sort_values(by=["kurtosis"],ascending=False).head(20)
r2["K"] = 1

# sort by skew
r3 = df4.loc[(df4["range"] >0.1) & (df4["num centile"]>2)].sort_values(by=["skew"],ascending=False).head(20)
r3["Sk"] = 1

#sort by SD
r4 = df4.loc[(df4["range"] >0.1) & (df4["num centile"]>2)].sort_values(by=["std"],ascending=False).head(20)
r4["SD"] = 1
r4

,chemical,chemical name,subpara,subpara name,num,num centile,count,median,min,max,range,std,kurtosis,skew,K
80,0906027G0,Vitamin B Compound,0906027,Vitamin B Compound,563923,10,207.0,1.00,0.83,1.00,0.17,0.01,126.43,-10.67,1
90,0601021X0,Tolbutamide,0601021,Sulfonylureas,6769,5,207.0,0.00,0.00,0.12,0.12,0.01,106.22,9.24,1
120,0501021B0,Cefadroxil,0501021,Cephalosporins,808,3,207.0,0.00,0.00,0.25,0.25,0.02,79.31,8.00,1
124,0902011U0,Potassium Chloride,0902011,Oral Potassium,32261,7,207.0,0.99,0.56,1.00,0.44,0.04,75.23,-7.39,1
139,0504010L0,Mefloquine Hydrochloride,0504010,Antimalarials,4353,4,207.0,0.00,0.00,0.32,0.32,0.04,64.97,8.16,1
152,0501120X0,Levofloxacin,0501120,Quinolones,3333,4,207.0,0.00,0.00,0.37,0.37,0.03,52.26,5.85,1
156,0906031C0,Ascorbic Acid,0906031,Vitamin C (Ascorbic Acid),35711,7,205.0,1.00,0.84,1.00,0.16,0.02,51.48,-7.02,1
160,0402010P0,Pericyazine,0402010,Antipsychotic Drugs,14674,6,207.0,0.00,0.00,0.24,0.24,0.03,49.53,6.99,1
174,0402010S0,Promazine Hydrochloride,0402010,Antipsychotic Drugs,26776,6,207.0,0.00,0.00,0.30,0.30,0.03,43.15,6.04,1
183,0410030B0,Buprenorph HCl/Naloxone HCl,0410030,Opioid Dependence,2582,4,188.0,0.00,0.00,0.33,0.33,0.04,38.66,5.83,1


In [50]:
# compile top 20 from each sort into a single output

rc = pd.merge(r1, r2, on=["chemical","chemical name","subpara","subpara name","num","num centile", "count","median","min","max","range", "std","kurtosis","skew"], how="outer")
rc = rc.merge(r3, on=["chemical","chemical name","subpara","subpara name","num","num centile", "count","median","min","max","range", "std","kurtosis","skew"], how="outer")
rc = rc.merge(r4, on=["chemical","chemical name","subpara","subpara name","num","num centile", "count","median","min","max","range", "std","kurtosis","skew"], how="outer").fillna(0)  

rc["score"] = rc["R"]+rc["K"]+rc["Sk"]+rc["SD"]
rc.sort_values(by=["score","range","kurtosis"],ascending=False)

,chemical,chemical name,subpara,subpara name,num,num centile,count,median,min,max,range,std,kurtosis,skew,R,K,Sk,SD,score
5,091101000,Other Amino Acid&Nutritional Agent Preps,0911010,Amino Acids & Nutritional Agents,2436,4,61.0,1.00,0.00,1.00,1.00,0.30,4.29,-2.43,1.0,0.0,0.0,1.0,2.0
8,0106050S0,Sodium Picosulfate,0106050,Bowel Cleansing Preparations,1292,3,49.0,0.80,0.00,1.00,1.00,0.29,1.54,-1.63,1.0,0.0,0.0,1.0,2.0
10,1301010D0,Cetomacrogol,1301010,Vehicles,23600,6,52.0,0.97,0.00,1.00,1.00,0.36,1.01,-1.66,1.0,0.0,0.0,1.0,2.0
12,0803020H0,Medroxyprogesterone Acetate,0803020,Progestogens,2393,3,76.0,0.79,0.00,1.00,1.00,0.31,0.45,-1.22,1.0,0.0,0.0,1.0,2.0
14,1501030H0,Hyoscine Hydrobromide,1501030,Antimuscarinic Drugs,4914,4,127.0,0.00,0.00,1.00,1.00,0.37,-0.16,1.26,1.0,0.0,0.0,1.0,2.0
15,1501030G0,Glycopyrronium Bromide,1501030,Antimuscarinic Drugs,14617,6,127.0,0.96,0.00,1.00,1.00,0.37,-0.20,-1.23,1.0,0.0,0.0,1.0,2.0
16,0803043N0,Octreotide Acetate,0803043,Somatostatin Analogues,1480,3,46.0,0.63,0.00,1.00,1.00,0.29,-0.30,-0.82,1.0,0.0,0.0,1.0,2.0
17,0906060L0,Menadiol Sodium Phosphate,0906060,Vitamin K,3798,4,118.0,0.67,0.00,1.00,1.00,0.32,-0.73,-0.83,1.0,0.0,0.0,1.0,2.0
18,0501090R0,Rifampicin,0501090,Antituberculosis Drugs,3528,4,104.0,0.59,0.00,1.00,1.00,0.31,-0.94,-0.57,1.0,0.0,0.0,1.0,2.0
19,0208010D0,Enoxaparin,0208010,Parenteral Anticoagulants,44616,7,206.0,0.34,0.00,1.00,1.00,0.37,-1.61,0.21,1.0,0.0,0.0,1.0,2.0


In [102]:
# most common paragraphs
rc.groupby(["subpara","subpara name"])["score"].agg(["count","sum"]).sort_values(by=["count","sum"],ascending=False)

,,count,sum
subpara,subpara name,,
0208010,Parenteral Anticoagulants,3,4.0
0402010,Antipsychotic Drugs,2,4.0
0601021,Sulfonylureas,2,4.0
1501030,Antimuscarinic Drugs,2,4.0
0410030,Opioid Dependence,2,3.0
0501090,Antituberculosis Drugs,2,3.0
0502050,Other Antifungals,2,3.0
0803020,Progestogens,2,3.0
0803043,Somatostatin Analogues,2,3.0


In [88]:
dftest = data2#.head(1000)
dftest
#df5 = pd.qcut(series, 10, labels=np.arange(1,11,1))
dftest["rank"] = dftest.groupby(["chemical"])["ratio"].rank(pct=True)
#dftest.groupby(["pct"])["ratio"].sum()
#dftest.loc[dftest["chemical"]=="0101010C0"].sort_values(by=["chemical","rank"])

x = dftest.loc[(dftest["rank"]<=0.05) | (dftest["rank"]>=0.95)]
x2 = x.groupby("pct")["rank"].count()
x2.sort_values(ascending=False)

####-------------------
#df_q = pd.DataFrame()
#for row in dftest:
   # df_q[row] = pd.qcut(dftest[row], 5, labels=list(range(5)))
    # this kicks out error ValueError: Bin edges must be unique: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.18248848,
       # 0.9124424 ])
        


pct
02W    190
03V    143
08V    137
10T    136
03Y    136
08M    127
99C    124
00N    121
03H    119
13T    119
07T    118
00L    113
00P    111
02R    111
03X    109
07G    107
07R    104
08C    102
06M    102
06W    101
00R     99
04C     98
09A     98
06V     97
04M     97
00C     97
03N     96
08Y     94
06D     94
10L     94
      ... 
99M     52
01A     52
05P     51
99J     51
13P     51
05N     51
99E     51
09E     51
11X     51
04V     50
03C     50
05R     49
10V     49
05J     47
09L     46
99N     46
01C     46
07Q     45
06F     43
05Q     43
09Y     42
07N     42
09G     41
08G     40
08A     39
09X     38
06K     36
06N     34
99H     32
07H     30
Name: rank, dtype: int64

In [93]:
# find top 10 most outliery CCGs for the combined list of interesting chemicals
ccgs = rc.merge(x, how = "left", on=["chemical"]).groupby("pct")["rank"].count()
ccgs.sort_values(ascending=False).head(10)

pct
10L    9
08M    8
02W    8
01H    8
07L    7
10X    7
08V    7
00T    7
02R    7
01D    7
Name: rank, dtype: int64

In [96]:
# investigate most outliery CCG
data2.loc[data2["pct"]=="10L"].merge(rc, how = "inner", on=["chemical"]).head()

########### notes
# this ccg measures 0 of 0 for several of these 

,pct,subpara_x,denom_subpara,chemical,num_x,denom,ratio,rank,chemical name,subpara_y,...,max,range,std,kurtosis,skew,R,K,Sk,SD,score
0,10L,1108020,70.0,1108020AG,0.0,0.0,0.000000,0.431319,Bromfenac,1108020,...,0.85,0.85,0.11,20.72,4.04,0.0,0.0,1.0,0.0,1.0
1,10L,1302010,4896.0,1302010Y0,0.0,0.0,0.000000,0.316425,Wool Alcohols,1302010,...,0.11,0.11,0.01,25.17,4.61,0.0,1.0,1.0,0.0,2.0
2,10L,1302020,444.0,1302020B0,35.0,444.0,0.078829,0.956522,Benzalkonium Chloride,1302020,...,0.37,0.37,0.04,38.65,5.17,0.0,1.0,1.0,0.0,2.0
3,10L,1306020,56.0,1306020J0,0.0,0.0,0.000000,0.470443,Isotretinoin,1306020,...,0.83,0.83,0.11,24.51,4.90,0.0,0.0,1.0,0.0,1.0
4,10L,1502010,1065.0,1502010C0,0.0,0.0,0.000000,0.364734,Bupivacaine Hydrochloride,1502010,...,0.21,0.21,0.02,34.70,4.85,0.0,1.0,1.0,0.0,2.0
